In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import TensorDataset
from tensorboardX import SummaryWriter
import time

from preprocess_pytorch import fetch_data
from sklearn.model_selection import train_test_split

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
NAME = "15000_pytorch_1v0_partialstate"
SAVE_EVERY = 10 # Save every N epochs

INPUT_SIZE = 59
NUM_EPOCHS = 30
BATCH_SIZE = 32
LEARNING_RATE = 0.0005

# Load Data

In [3]:
X, y = fetch_data(['data/15000_partialstate_1v0.log'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0, random_state=42)
y_train_actions, y_train_parameters = y_train[:,0].astype(np.long), y_train[:,1:]
y_test_actions, y_test_parameters = y_test[:,0].astype(np.long), y_test[:,1:]

In [4]:
num_train_data = X_train.shape[0]
num_test_data = X_test.shape[0]

In [5]:
# Dataset
train_dataset = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train_actions), torch.from_numpy(y_train_parameters))
test_dataset = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test_actions), torch.from_numpy(y_test_parameters))

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=BATCH_SIZE, 
                                          shuffle=False)

# Train

In [10]:
# DASH TURN TACKLE KICK 2 1 1 2

# Fully connected neural network with one hidden layer
from pytorch_model import NeuralNet
    
def get_test_loss(model, criterion_actions, criterion_parameters, test_loader, num_test_data):
    return 0, 0 # TODO REMOVE
    test_loss_actions = 0
    test_loss_parameters = 0
    with torch.no_grad():
        for X, y_actions, y_parameters in test_loader:
            X = X.to(device)
            y_actions = y_actions.to(device)
            y_parameters = y_parameters.to(device)
            outputs_actions, output_parameters = model(X)
            loss_actions = criterion_actions(outputs_actions, y_actions)            
            relevance_map = y_parameters != y_parameters
            y_parameters[relevance_map] = 0
            output_parameters[relevance_map] = 0
            
            loss_parameters = criterion_parameters(output_parameters, y_parameters) / 2 / 400
            
            test_loss_actions += loss_actions.item()
            test_loss_parameters += loss_parameters.item()
    return test_loss_actions / num_test_data, test_loss_parameters / num_test_data

In [11]:
model = NeuralNet().to(device).double()
# Loss and optimizer
criterion_actions = nn.CrossEntropyLoss(size_average=False)
criterion_parameters = nn.MSELoss(size_average=False) # MSE Sum
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

writer = SummaryWriter() # Tensorboard

In [12]:
start_time = time.time()
time.sleep(5)
x = time.time() - start_time # seconds

In [ ]:
# Train the model
total_step = len(train_loader)
update_interval = total_step // 3
start_time = time.time()
for epoch in range(NUM_EPOCHS):
    train_loss_actions = 0
    train_loss_parameters = 0
    for i, (X, y_actions, y_parameters) in enumerate(train_loader):  
        # Move tensors to the configured device
        X = X.to(device)
        y_actions = y_actions.to(device)
        y_parameters = y_parameters.to(device)
        
        N = X.shape[0] # num of data
        
        # Forward pass
        output_actions, output_parameters = model(X)
        loss_actions = criterion_actions(output_actions, y_actions)
        
        # Figure out relevance using nans
        relevance_map = y_parameters != y_parameters
        y_parameters[relevance_map] = 0
        output_parameters[relevance_map] = 0
        
        loss_parameters = criterion_parameters(output_parameters, y_parameters) / 2 / 400
        loss = (loss_actions + loss_parameters) / N
        
        train_loss_actions += loss_actions
        train_loss_parameters += loss_parameters
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % update_interval == 0:
            print('Epoch [{}/{}], Iteration [{}/{}], Time Elapsed: {:.4f} s'.format(
                epoch+1, NUM_EPOCHS, i + 1, total_step, time.time() - start_time))
        
    train_loss_actions /= num_train_data
    train_loss_parameters /= num_train_data
    train_loss_total = train_loss_actions + train_loss_parameters
    
    test_loss_actions, test_loss_parameters = get_test_loss(
        model, criterion_actions, criterion_parameters, test_loader, num_test_data)
    test_loss_total = test_loss_actions + test_loss_parameters
    
    writer.add_scalar('total_loss/1:training_loss', train_loss_total, epoch + 1)
    writer.add_scalar('total_loss/2:test_loss', test_loss_total, epoch + 1)
    
    writer.add_scalar('action_loss/1:training_loss', train_loss_actions, epoch + 1)
    writer.add_scalar('action_loss/2:test_loss', test_loss_actions, epoch + 1)
    
    writer.add_scalar('parameters_loss/1:training_loss', train_loss_parameters, epoch + 1)
    writer.add_scalar('parameters_loss/2:test_loss', test_loss_parameters, epoch + 1)
    
    if (epoch + 1) % SAVE_EVERY == 0:
        filename = './models/%s_%d.model' % (NAME, epoch)
        torch.save(model.state_dict(), filename)
        print("File %s created" % filename)
    
    print('Epoch [{}/{}], Loss: {:.4f} ({:.4f} + {:.4f}), Test Loss: {:.4f} ({:.4f} + {:.4f})'.format(
        epoch+1, NUM_EPOCHS, 
        train_loss_total, train_loss_actions, train_loss_parameters,
        test_loss_total, test_loss_actions, test_loss_parameters))
    print("Time Elapsed: {:.4f} mins".format((time.time() - start_time) / 60.0))
    
filename = './models/%s_FINAL.model' % (NAME)
torch.save(model.state_dict(), filename)
print("File %s created" % filename)

Epoch [1/30], Iteration [11601/34805], Time Elapsed: 37.8670 s
Epoch [1/30], Iteration [23202/34805], Time Elapsed: 75.5659 s
Epoch [1/30], Iteration [34803/34805], Time Elapsed: 113.3066 s
Epoch [1/30], Loss: 0.3234 (0.1146 + 0.2087), Test Loss: 0.0000 (0.0000 + 0.0000)
Time Elapsed: 1.8979 mins
Epoch [2/30], Iteration [11601/34805], Time Elapsed: 151.7009 s
Epoch [2/30], Iteration [23202/34805], Time Elapsed: 189.2841 s
Epoch [2/30], Iteration [34803/34805], Time Elapsed: 226.9307 s
Epoch [2/30], Loss: 0.1916 (0.0743 + 0.1173), Test Loss: 0.0000 (0.0000 + 0.0000)
Time Elapsed: 3.7901 mins
Epoch [3/30], Iteration [11601/34805], Time Elapsed: 265.1433 s
Epoch [3/30], Iteration [23202/34805], Time Elapsed: 302.8361 s
Epoch [3/30], Iteration [34803/34805], Time Elapsed: 340.6153 s
Epoch [3/30], Loss: 0.1590 (0.0649 + 0.0941), Test Loss: 0.0000 (0.0000 + 0.0000)
Time Elapsed: 5.6857 mins
Epoch [4/30], Iteration [11601/34805], Time Elapsed: 378.9511 s
Epoch [4/30], Iteration [23202/34805],

In [ ]:
np.unique(y_train[:,0], return_counts=True)

In [ ]:
np.unique(y_test[:,0], return_counts=True)

In [ ]:
# r_t = torch.from_numpy(r).float().unsqueeze(0)
# model(r_t.cuda().double())